# Лабораторная работа №5

## Импорт библеотек

In [39]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.vocab import build_vocab_from_iterator
from transformers import BertTokenizer, BertForSequenceClassification, AdamW


import pandas as pd
import numpy as np

import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords


from tqdm.auto import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Функция Обучения

In [2]:
def train(model, optimizer, criterion, n_epochs, train_loader, test_loader):

  loss_train = []
  accuracy_train = []

  for epoch in range(n_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f"Training epoch {epoch + 1}/{n_epochs}"):
        inputs = batch["input"]
        labels = batch["label"]
        output = model(inputs)

        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
      for batch in tqdm(test_loader, desc=f"Testing epoch {epoch + 1}/{n_epochs}"):
        inputs = batch["input"]
        labels = batch["label"]
        output = model(inputs)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    accuracy_train.append(test_accuracy)
    print('Epoch [{}/{}], Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch + 1, n_epochs, loss.item(), test_accuracy * 100))
    loss_train.append(loss.item())


## Google Drive if need

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Задание 1. Изучите технологии attention и архитектуры нейронных сетей трансформеров.

## Задание 2. Примените один из трансформеров, например BERT к задаче классификации отзывов клиентов. Сравните полученные результаты с классическими методами машинного обучения, с RNN. Сделайте выводы.

### Dataset

In [14]:
df_reviews = pd.read_csv('/content/drive/MyDrive/NLP_FU/lem_500kdata.csv', index_col = 0)
df_reviews.dropna(inplace = True)
df_reviews

,rating,text
0,3,московский квартал шумно лето ночь дикий гонка...
1,5,замечательный сеть магазин общий хороший ассор...
2,1,знать смутить коготь дать правило удивить хоте...
3,4,хороший условие аренда дружелюбный персонално ...
4,5,топ мастер ангелина топ смысл немного волноват...
...,...,...
499994,5,хороший способ избежать автобусный пытка элект...
499995,4,охрана кривая добрый дверь закрываться автомат...
499996,4,сравнение многий современный платформа эпоха с...
499998,5,семья отличный место рекомендовать это настоящ...


### Balance classes

In [21]:
min_class_size  = df_reviews['rating'].value_counts().min()

df_reviews['rating'].value_counts()

5    389674
4     41114
1     34297
3     21651
2     12071
Name: rating, dtype: int64

In [25]:
df_reviews = pd.concat([group.sample(min_class_size) for _, group in df_reviews.groupby('rating')])

In [26]:
df_reviews

,rating,text
232095,1,плохой пункт королева последний время терминал...
169402,1,ужасный ситуация детский стоматология просто к...
283945,1,место малолетка наверно место спаивать малолет...
65350,1,покупать фрезерный станок осень прошлое год эт...
436450,1,звезда руководство отель обращать внимание отз...
...,...,...
398224,5,сотрудник персонал достаточно дружелюбный поси...
190575,5,приятный новый магазин удобный парковка достат...
413532,5,большой торговый центр недалеко метро войковск...
166771,5,компания знакомый годвсе это время отправлять ...


### DeepPavlov

In [7]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased-sentence", num_labels=5)

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Tokenizer

In [27]:
texts = df_reviews['text'].values

print(texts[:5])


tokenized_inputs = []
for text in tqdm(texts, desc="Tokenizing"):
    inputs = tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, max_length=128)
    tokenized_inputs.append(inputs)

['плохой пункт королева последний время терминал оплата карта требовать наличный перевод карта идти'
 'ужасный ситуация детский стоматология просто кромешный ужас'
 'место малолетка наверно место спаивать малолетка муха проблема'
 'покупать фрезерный станок осень прошлое год это время работа радость никакой испытать останавливаеиться время рабочий процесс повторный нажатие идти траектория вырезать одинаклвый заготовка отверстие резать смещение мм отверстие программа далее снатий половина поверхность изделие резать погрызать широкий мм должный программа далее резать нужный размер мм сделать установка убирать стружка весь день должный стоять пылесос позициониоовать упаковка который приходить стол полный бред перекосить ящик тонюсенький фанера ровный уровень вообще хороший вырезать стоя сварить стол практически идеально откорректировать разный часть стол стол резать непрорез мм это установленный плюс погрешность сотрудник служба поддержка отвечать быстро материал испортить ответ ремень ро

Tokenizing:   0%|          | 0/60355 [00:00<?, ?it/s]

Save on Google Drive

In [31]:
with open('/content/drive/MyDrive/NLP_FU/tokenized_inputs.pkl', 'wb') as f:
    pickle.dump(tokenized_inputs, f)

Read from Google Drive

In [34]:
# with open('/content/drive/MyDrive/NLP_FU/tokenized_inputs.pkl', 'rb') as f:
#     tokenized_inputs = pickle.load(f)

### Dataset

In [43]:
dataset = TensorDataset(tokenized_inputs, torch.tensor(df_reviews['rating'].values, dtype = torch.long))
train_size = len(dataset) * 0.8
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# # Создание DataLoader для эффективной обработки данных во время обучения
# train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

AttributeError: ignored

In [64]:
class MyDataset(Dataset):

    def __init__(self, all_reviews, labels):
        self.all_reviews = all_reviews
        self.labels = labels

    def __getitem__(self, ids):

        return {
            'input_ids' : torch.tensor(self.all_reviews[ids]['input_ids'], dtype=torch.long),
            'token_type_ids' : torch.tensor(self.all_reviews[ids]['token_type_ids'], dtype=torch.long),
            'attention_mask' : torch.tensor(self.all_reviews[ids]['attention_mask'], dtype=torch.long),
            'labels' : torch.tensor(self.labels.values[ids], dtype=torch.long)
        }

    def __len__(self):
        return len(self.all_reviews)

In [65]:
dataset = MyDataset(tokenized_inputs, df_reviews['rating'])
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [67]:
train_dataset[123]

<ipython-input-64-c16de48565e4>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids' : torch.tensor(self.all_reviews[ids]['input_ids'], dtype=torch.long),
<ipython-input-64-c16de48565e4>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids' : torch.tensor(self.all_reviews[ids]['token_type_ids'], dtype=torch.long),
<ipython-input-64-c16de48565e4>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask' : torch.tensor(self.all_reviews[ids]['attention_mask'], dtype=torch.long),


{'input_ids': tensor([[   101,  63162,  10405,  12886,  13390,  59677,  36751,    896,  19101,
            3998,  12620,   1671,  28204,   3998,  13555,  20821,  98957,  79351,
           17947,  11333,   7079,  33466,  16743,  18040,  14575,  15636,  12904,
            7457,   5883,  30444,  34118,  14327,  21326,  21715,   2603,  51314,
            8056,  21867,  14934,  16743,  39578,   2226,  65268,  15928,  10803,
           28587,  12993,   9728,  22571,   1523,  52027,  10803,  28973,   3998,
           13123,  22571,  51314,   8056,  12313,   7433,   3579,   2195,  10796,
           23546,  19539,   7805,  33308,  12313,  15636,  12886,  66852, 102730,
            4099,  14935,   1662,    102,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,      0,
   

### DataLoader

In [68]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

### Train

In [70]:
n_epochs = 3

optimizer = AdamW(model.parameters(), lr=1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [71]:
loss_train = []
accuracy_train = []

for epoch in range(n_epochs):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Training epoch {epoch + 1}/{n_epochs}"):


        optimizer.zero_grad()
        input_ids, token_type_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc=f"Testing epoch {epoch + 1}/{n_epochs}"):
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
            total += (predictions == labels).sum().item()
            correct += len(labels)

    accuracy = total / correct
    print(f"Accuracy on test set: {accuracy}")

Training epoch 1/3:   0%|          | 0/6036 [00:00<?, ?it/s]

<ipython-input-64-c16de48565e4>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids' : torch.tensor(self.all_reviews[ids]['input_ids'], dtype=torch.long),
<ipython-input-64-c16de48565e4>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids' : torch.tensor(self.all_reviews[ids]['token_type_ids'], dtype=torch.long),
<ipython-input-64-c16de48565e4>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask' : torch.tensor(self.all_reviews[ids]['attention_mask'], dtype=torch.long),


AttributeError: ignored

In [ ]:
def train(model, optimizer, criterion, n_epochs, train_loader, test_loader):

  loss_train = []
  accuracy_train = []

  for epoch in range(n_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f"Training epoch {epoch + 1}/{n_epochs}"):
        inputs = batch["input"]
        labels = batch["label"]
        output = model(inputs)

        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
      for batch in tqdm(test_loader, desc=f"Testing epoch {epoch + 1}/{n_epochs}"):
        inputs = batch["input"]
        labels = batch["label"]
        output = model(inputs)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    accuracy_train.append(test_accuracy)
    print('Epoch [{}/{}], Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch + 1, n_epochs, loss.item(), test_accuracy * 100))
    loss_train.append(loss.item())


## Задание 3. Примените один из трансформеров, например BERT, к задаче генерации англоязычного и русскоязычного текстов. Сравните результаты с LSTM. Сделайте выводы.

## Задание 4. Примените один из трансформеров, например BERT, к задаче машинного перевода.